In [28]:
import os
from getpass import getpass
from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
import shutil
from tqdm.auto import tqdm
import ast
import pybboxes as pbx

import ultralytics
from ultralytics import YOLO

import random

In [ ]:
os.environ['KAGGLE_USERNAME'] = input()
os.environ['KAGGLE_KEY'] = getpass()

api = KaggleApi()
api.authenticate()

In [2]:
!kaggle datasets download -d vbookshelf/v2-balloon-detection-dataset
!unzip /workspace/object-detection-balloons/v2-balloon-detection-dataset.zip -d ./balloons
!rm -rf /workspace/object-detection-balloons/v2-balloon-detection-dataset.zip

In [9]:
csv_url = "/workspace/object-detection-balloons/balloons/balloon-data.csv"
balloon_df = pd.read_csv(csv_url)
print(balloon_df.shape)
balloon_df.head()

(74, 5)


,fname,height,width,bbox,num_balloons
0,34020010494_e5cb88e1c4_k.jpg,1536,2048,"[{'xmin': 994, 'ymin': 619, 'xmax': 1445, 'yma...",1
1,25899693952_7c8b8b9edc_k.jpg,1365,2048,"[{'xmin': 135, 'ymin': 115, 'xmax': 811, 'ymax...",1
2,24362039530_b151b41a52_k.jpg,2048,1536,"[{'xmin': 579, 'ymin': 487, 'xmax': 920, 'ymax...",1
3,18849792632_aad23ad513_k.jpg,2048,1536,"[{'xmin': 534, 'ymin': 365, 'xmax': 871, 'ymax...",10
4,17178818589_16e58fc1e5_k.jpg,1536,2048,"[{'xmin': 748, 'ymin': 195, 'xmax': 1425, 'yma...",7


In [45]:
random.seed(2077)

balloon_df["is_val"] = [random.uniform(0, 1) <= 0.2 for _ in range(len(balloon_df))]

In [48]:
balloon_df.loc[balloon_df["is_val"] == True].shape

(16, 6)

In [13]:
# os.rename("/workspace/object-detection-balloons/balloons/images", "/workspace/object-detection-balloons/balloons/train")
# os.mkdir("/workspace/object-detection-balloons/balloons/images")
# shutil.move("/workspace/object-detection-balloons/balloons/train", "/workspace/object-detection-balloons/balloons/images/train")

os.makedirs("/workspace/object-detection-balloons/balloons/labels/train", exist_ok=True)

In [51]:
img_val_dir = "/workspace/object-detection-balloons/balloons/images/val"
os.mkdir(img_val_dir)

In [56]:
lbs_val_dir = "/workspace/object-detection-balloons/balloons/labels/val"
os.mkdir(lbs_val_dir)

In [54]:
for i, row in balloon_df.loc[balloon_df["is_val"] == True].iterrows():
    src_path = os.path.join("/workspace/object-detection-balloons/balloons/images/train", row["fname"])
    dst_path = os.path.join("/workspace/object-detection-balloons/balloons/images/val", row["fname"])
    shutil.move(src_path, dst_path)

In [58]:
for i, row in balloon_df.loc[balloon_df["is_val"] == True].iterrows():
    src_path = os.path.join("/workspace/object-detection-balloons/balloons/labels/train", row["fname"].replace(".jpg", ".txt"))
    dst_path = os.path.join("/workspace/object-detection-balloons/balloons/labels/val", row["fname"].replace(".jpg", ".txt"))
    shutil.move(src_path, dst_path)

In [14]:
def create_yolo_labels(row):
    dir = "/workspace/object-detection-balloons/balloons/labels/train/"
    bboxes = ast.literal_eval(row["bbox"])
    for bbox in bboxes:
        bbox = bbox.values()
        w, h = row["width"], row["height"]
        bbox_yolo = pbx.convert_bbox(bbox, from_type="voc", to_type="yolo", image_size=(w, h))
        bbox_yolo = " ".join([str(bb) for bb in bbox_yolo])
        file_path = dir + row["fname"].replace(".jpg", ".txt")
        with open(file_path, "a") as f:
            f.write(f"0 {bbox_yolo}\n")

# create_yolo_labels(balloon_df.iloc[0])

In [15]:
for i, row in tqdm(balloon_df.iterrows()):
    create_yolo_labels(row)
    

0it [00:00, ?it/s]

In [16]:
ultralytics.checks()

Ultralytics YOLOv8.0.164 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Setup complete ✅ (48 CPUs, 125.7 GB RAM, 1.1/20.0 GB disk)


In [24]:
yolo_base = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

In [60]:
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
yaml_path = "/workspace/object-detection-balloons/balloons/balloons.yaml"

# Use the model
results = model.train(data=yaml_path, epochs=50)  # train the model

Ultralytics YOLOv8.0.164 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/workspace/object-detection-balloons/balloons/balloons.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, box

  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

In [21]:
img_path_base = "/workspace/object-detection-balloons/balloons/images/train"
img_paths = [os.path.join(img_path_base, fname) for fname in os.listdir(img_path_base)]
img_paths[:3]

['/workspace/object-detection-balloons/balloons/images/train/10464445726_6f1e3bbe6a_k.jpg',
 '/workspace/object-detection-balloons/balloons/images/train/12037308314_e16fb3a0f7_k.jpg',
 '/workspace/object-detection-balloons/balloons/images/train/120853323_d4788431b9_b.jpg']

In [27]:
preds = model.predict(img_paths[10:20], save=True) 


0: 640x640 3 balloons, 1: 640x640 8 balloons, 2: 640x640 3 balloons, 3: 640x640 1 balloon, 4: 640x640 11 balloons, 5: 640x640 2 balloons, 6: 640x640 5 balloons, 7: 640x640 7 balloons, 8: 640x640 1 balloon, 9: 640x640 1 balloon, 11.3ms
Speed: 1.5ms preprocess, 1.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict4
